In [76]:
import nba_py
import datetime
import nba_py.game
import nba_py.player
import nba_py.team
import pandas as pd
import functools

today = datetime.date.today()
tomorrow = today + datetime.timedelta(days = 1)
someday = datetime.date(2017, 1, 1)

In [3]:
def get_games(date):
    '''
    return a pandas.Series of all the games available on this day.
    '''
    return nba_py.Scoreboard(month = date.month, 
                             day = date.day, 
                             year = date.year, 
                             league_id = '00', 
                             offset = 0).available()['GAME_ID']

get_games(someday)

0    0021600510
1    0021600509
2    0021600508
3    0021600511
4    0021600512
Name: GAME_ID, dtype: object

In [71]:
def get_teams(games):
    '''
    return a pandas.DataFrame including every games' home team id and away team id.
    '''
    teams = [nba_py.game.BoxscoreSummary(g).game_summary().loc[0, ['HOME_TEAM_ID', 'VISITOR_TEAM_ID']] 
             for g in games]
    return pd.DataFrame(teams, index = [g for g in games])

teams = get_teams(get_games(someday))
teams

,HOME_TEAM_ID,VISITOR_TEAM_ID
0021600510,1610612754,1610612753
0021600509,1610612748,1610612765
0021600508,1610612737,1610612759
0021600511,1610612750,1610612757
0021600512,1610612747,1610612761


In [101]:
def get_players(teams):
    '''
    return a pandas.DataFrame of all the teams' players.
    teams must be a pandas.DataFrame created by the function get_teams().
    '''
    player_list = nba_py.player.PlayerList().info()
    all_teams = teams['HOME_TEAM_ID'].append(teams['VISITOR_TEAM_ID'])
    players = functools.reduce(pd.Series.append, 
                                [nba_py.team.TeamPlayers(team).season_totals()['PLAYER_ID'] 
                                 for team in all_teams])
    players = players.to_frame()
    players['TEAM_ID'] = [int(player_list[player_list['PERSON_ID'] == p]['TEAM_ID']) 
                          for p in players['PLAYER_ID']]
    players = players[players['TEAM_ID'].isin(all_teams.values)]
    players['AGAINST_TEAM_ID'] = [int(teams[teams['HOME_TEAM_ID'] == t]['VISITOR_TEAM_ID'] 
                              if t in teams['HOME_TEAM_ID'].values 
                              else teams[teams['VISITOR_TEAM_ID'] == t]['HOME_TEAM_ID']) 
                          for t in players['TEAM_ID']]
    return players
    

get_players(get_teams(get_games(someday)))

,PLAYER_ID,TEAM_ID,AGAINST_TEAM_ID
0,201166,1610612754,1610612753
1,2744,1610612754,1610612753
2,101139,1610612754,1610612753
3,1627777,1610612754,1610612753
4,203922,1610612754,1610612753
5,201952,1610612754,1610612753
6,1626202,1610612754,1610612753
7,202338,1610612754,1610612753
8,202362,1610612754,1610612753
9,202730,1610612754,1610612753


In [5]:
nba_py.player.PlayerLastNGamesSplits('201939').last10().columns

Index(['GROUP_SET', 'GROUP_VALUE', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM',
       'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD',
       'PTS', 'PLUS_MINUS', 'DD2', 'TD3', 'GP_RANK', 'W_RANK', 'L_RANK',
       'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK',
       'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK',
       'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK', 'REB_RANK', 'AST_RANK',
       'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK', 'PFD_RANK',
       'PTS_RANK', 'PLUS_MINUS_RANK', 'DD2_RANK', 'TD3_RANK', 'CFID',
       'CFPARAMS'],
      dtype='object')

In [6]:
play_stats_cols = ['GROUP_VALUE', 'GROUP_SET', 'GP', 
                   'MIN', 'PTS', 'AST', 'OREB', 'DREB', 'STL', 'BLK', 'TOV', 
                   'FGM', 'FGA', 'FG3M']

In [7]:
nba_py.player.PlayerLastNGamesSplits('201939').last10()[play_stats_cols]

,GROUP_VALUE,GROUP_SET,GP,MIN,PTS,AST,OREB,DREB,STL,BLK,TOV,FGM,FGA,FG3M
0,2016-17,Last 10 Games,10,33.2,28.4,8.6,0.8,3.8,2.4,0.4,3.2,9.7,19.4,5.2
